## Visualizing ECOSTRESS LSTE and ESI

In the last notebook, we search for and downloaded ECOSTRESS LSTE and ESI data products for our study region (Rocky Mountain National Park; RMNP). In this notebook, we will extract data from the NetCDF files we downloaded and perform some subsetting to get our granules to an analysis-ready stage.

NetCDF (".h5") files are ... add a description

Notebook Goals:

- Open and extract data from ".h5" files downloaded from NASA Earthdata
- Create an interactive visualization to look at data coverage
- Create a simple time-series plot between our three dates
- Create a continuous mosaic for our study region
- Export GeoTIFF files of LST and ESI for further analysis

### Step 1. Setup the notebook

As last time, we need to first set up the notebook with the required packages and environment variables.

In [17]:
"""
Load the necessary packages and set environment variables
"""

# Import packages
import os, shutil, time, glob, warnings
import folium
import earthaccess
import pandas as pd
import geopandas as gpd
import rasterio as rio
import rioxarray as rxr
import h5py
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt
from affine import Affine

# Projection information
geog = 'EPSG:4326'  # Geographic projection
prj = 'EPSG:5070'  # Projected coordinate system- WGS 84 NAD83 UTM Zone 13N

# File path information
datadir = '/data-store/iplant/home/shared/esiil/HYR_SENSE/'

# File path information
print("Success")

Success


### Step 2. Load the ECOSTRESS Data

Load the data files we downloaded last time. Check for data quality.


In [18]:
ecodir = '/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO_L2G_LSTE/'

def list_files(path, ext, recursive):
    """
    List files of a specific type in a directory or subdirectories
    """
    if recursive is True:
        return glob.glob(os.path.join(path, '**', '*{}'.format(ext)), recursive=True)
    else:
        return glob.glob(os.path.join(path, '*{}'.format(ext)), recursive=False)

# Get a list of .nc files
nc_files = list_files(ecodir,"*.h5",recursive=True)
print(nc_files)

['/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO_L2G_LSTE/ECOv002_L2G_LSTE_22291_005_20220611T170655_0700_01.h5', '/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO_L2G_LSTE/ECOv002_L2G_LSTE_22306_006_20220612T161753_0700_01.h5', '/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO_L2G_LSTE/ECOv002_L2G_LSTE_22306_007_20220612T161845_0700_01.h5', '/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO_L2G_LSTE/ECOv002_L2G_LSTE_23070_006_20220731T210132_0700_01.h5', '/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO_L2G_LSTE/ECOv002_L2G_LSTE_23146_005_20220805T183439_0700_01.h5', '/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO_L2G_LSTE/ECOv002_L2G_LSTE_23177_009_20220807T183406_0700_01.h5', '/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO_L2G_LSTE/ECOv002_L2G_LSTE_23192_008_20220808T174618_0700_01.h5', '/home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO_L2G_LSTE/ECOv002_L2G_LSTE_24143_008_20221008T172738_0700_01.h5']


In [19]:
# Function to open and print NetCDF items
def read_h5_file(fp):
    
    def print_attrs(name, obj):
        print(name)
        for key, val in obj.attrs.items():
            print(f"    {key}: {val}")
            
    with h5py.File(fp, 'r') as f:
        print(f'Contents of {fp}')
        f.visititems(print_attrs)

# Open the first file to examine the contents
read_h5_file(nc_files[0])

Contents of /home/jovyan/HYR-SENSE/data/Drought-FireRisk/ECO_L2G_LSTE/ECOv002_L2G_LSTE_22291_005_20220611T170655_0700_01.h5
HDFEOS
HDFEOS/ADDITIONAL
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/ProductMetadata
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/ProductMetadata/AncillaryNWP
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/ProductMetadata/BandSpecification
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/ProductMetadata/CloudMaxTemperature
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/ProductMetadata/CloudMeanTemperature
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/ProductMetadata/CloudMinTemperature
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/ProductMetadata/CloudSDevTemperature
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/ProductMetadata/Emis1GoodAvg
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/ProductMetadata/Emis2GoodAvg
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/ProductMetadata/Emis3GoodAvg
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/ProductMetadata/Emis4GoodAvg
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/ProductMetadata/Emis5GoodAvg
HDFEOS/ADDITIONAL/FILE_ATTRIBUTES/Produ

We can see that the ".h5" file is made up of quite a lot of information! But we are interested in retrieving the LST grid so let's focus on that. We can now modify our function to extract the LST grid and append it to a list. In the NetCDF structure, we can see a number of data fields at the end. The LST data is stored within "HDFEOS/GRIDS/ECO_L2G_LSTE_70m/Data Fields/LST".

In [ ]:
## Modified read function for opening a data file
def read_h5_file(fp, dataset='LST', print_contents=False):
    
    def print_attrs(name, obj):
        print(name)
        for key, val in obj.attrs.items():
            print(f"    {key}: {val}")
    
    with h5py.File(fp, 'r') as f:
        if print_contents == True:
            print(f'Contents of {fp}')
            f.visititems(print_attrs)
        else:
            # Extracting the LST dataset
            lst_data = f['HDFEOS/GRIDS/ECO_L2G_LSTE_70m/Data Fields/LST'][:]
            # Extracting the fill value
            fill_value = f['HDFEOS/GRIDS/ECO_L2G_LSTE_70m/Data Fields/LST'].attrs['_Fillvalue']
            
            # Converting to xarray DataArray and setting fill value to NaN
            lst_data = xr.DataArray(lst_data)
            lst_data = lst_data.where(lst_data != fill_value, np.nan)

            # Set to a spatial raster layer
            transform = Affine(
                f['HDFEOS/GRIDS/ECO_L2G_LSTE_70m/Projection/GeoTransform'][0],
                f['HDFEOS/GRIDS/ECO_L2G_LSTE_70m/Projection/GeoTransform'][1],
                f['HDFEOS/GRIDS/ECO_L2G_LSTE_70m/Projection/GeoTransform'][2],
                f['HDFEOS/GRIDS/ECO_L2G_LSTE_70m/Projection/GeoTransform'][3],
                f['HDFEOS/GRIDS/ECO_L2G_LSTE_70m/Projection/GeoTransform'][4],
                f['HDFEOS/GRIDS/ECO_L2G_LSTE_70m/Projection/GeoTransform'][5]
            )

            # Set CRS and affine transform to the DataArray
            lst_data = lst_data.rio.write_crs(prj) # EPSG:5070
            lst_data.rio.write_transform(transform, inplace=True)

    return lst_data
    
# Extract the LST grids from each file
# Crop the grids to our RMNP study region
for h5f in nc_files:
    # Read LST data from .h5 file
    lst_data = read_h5_file(h5f)
    print(lst_data)

### Step 3. Interactive Maps of LSTE and ESI

We can use the Folium package to create interactive maps for our study region and the cleaned data from above

### Step 4. Export TIF files
